In [ ]:
import data_import
import os
import matplotlib.pyplot as plt
import numpy as np
import pickle
from scipy.ndimage.filters import gaussian_filter


plt.rcParams.update({'font.size': 20})
plt.rc('figure', figsize=(20, 10))

In [ ]:
def exec(dt=1e-2, output_dt=1e-1, tf=60*3, vz=3, nh = 10):

    command = f"java -cp target/Zombies-1.0.jar \
                -Ddt={dt} -DoutputDt={output_dt} -Dtf={tf} \
                -Dvz={vz} -Dnh={nh} \
                Main"
    proc = os.popen(command)
    proc.readlines()
    proc.close()
    data = data_import.Data("zombies.txt")
    return data

In [ ]:
results_b = {}
nh = [2, 10, 40, 80, 140, 200, 260, 320]
iterations = 10

if os.path.isfile("results_b.pickle"):
    results_b = pickle.load(open("results_b.pickle", "rb"))
else:


    for n in nh:
        res_iterations = {}
        for iteration in range(iterations):
            data = exec(nh=n, dt=1e-2, output_dt=1)
            time = []
            zcount = []
            res = {}
            for t, z, h, data in data:
                time.append(t)
                zcount.append(z)
            res["time"] = time
            res["zcount"] = zcount
            res_iterations[iteration] = res
        results_b[n] = res_iterations
            



In [ ]:
iteration = 1
for n in reversed(nh):
    y = np.array(results_b[n][iteration]["zcount"])/(n+1)
    #if(y[0] == y[-1]): continue
    y = gaussian_filter(y, sigma=3)
    plt.plot(results_b[n][iteration]["time"], y , label=f"{n} humanos")
plt.legend()
plt.xlabel("Tiempo (s)")
plt.ylabel("Fraccion de zombies (z/N)")
plt.show()

In [ ]:
sigma = 3
iter = 5
for n in reversed(nh):
    y = np.array(results_b[n][iter]["zcount"])
   # if y[0] == y[-1]: continue
    y = gaussian_filter(y, sigma, order=1, mode='nearest')
    #y = np.diff(results_b[n][iter]["zcount"], prepend=1)
    dt = results_b[n][iter]["time"][1] - results_b[n][iter]["time"][0]
    y = np.array(y)/dt
    plt.plot(results_b[n][iter]["time"], y, label=f"{n} humanos")
plt.legend()
plt.ylabel("Velocidad de contagio (z/s)")
plt.xlabel("Tiempo (s)")
plt.show()


In [ ]:
means = []
errors = []
for n in nh:
    values = []
    for iteration in range(iterations):
        delta = results_b[n][iteration]["time"][1]-results_b[n][iteration]["time"][0]
        dif =np.diff(results_b[n][iteration]["zcount"])/delta
        values.append(np.mean(dif))
    means.append(np.mean(values))
    errors.append(np.std(values))
plt.errorbar(nh, means, yerr=errors, fmt="o-", capsize=10)
plt.xlabel("Numero de humanos")
plt.ylabel("Velocidad de contagio media(z/s)")
plt.grid()
plt.xticks(nh)
plt.show()

In [ ]:
means = []
errors = []
percentage = 100
for n in nh:
    values = []
    for iteration in range(iterations):
        time = 0
        count = results_b[n][iteration]["zcount"]
        times = results_b[n][iteration]["time"]
        for t, z in zip(times,count):
            if z/n > percentage/100:
                time = t
                break
        
        values.append(t)
    means.append(np.mean(values))
    errors.append(np.std(values)/np.sqrt(len(values)))
plt.errorbar(nh, means, yerr=errors, fmt="o-", capsize=10)
plt.xlabel("Numero de humanos")
plt.ylabel(f"Tiempo hasta Fz de {percentage}% (s)")
plt.grid()
plt.xticks(nh)
plt.show()

In [ ]:
pickle.dump(results_b, open("results_b.pickle", "wb"))